# Partition_Apriori
The partition Apriori is supposed to be faster than original Apriori. However mine is slower. I think the reason is probably because I called Apriori code to get local frequent itemset.
I tried many values of partition lenth, it looks 1000 get best excution time

In [9]:
# slipt dataset D to N number of partitions and store them in P
def partition_dataset(D, N, len_of_partition):
    P = []
    i = 1
    for i in range(N):
        P.append(D[(i - 1) * len_of_partition: i * len_of_partition])
        i += 1
    return P


# phase1 generate local frequent items to L
def phase1(P, min_sup):
    L = []
    for partition in P:
        local_min_sup = min_sup * len(partition)/total_transaction #equally scale down the local_min_sup from global one
        Lp = apriori (partition, local_min_sup) # use Apriori code block, this causes longer excution time
        L.append(Lp)
    return L


# combine all local frequent item in L to Ckg
def combine(L, P):
    Ckg = []
    k = 0
    for j in range(len(P)):
        while k < len(L[j]) and len(L[j][k]) != 0:
            Ckg.append(L[j][k])
            k += 1

    return Ckg


# find global frequent items from Ckg
def phase2(P, Ckg):
    measure_support = defaultdict(int)
    for partition in P:
        for ck in Ckg:
            for freq_item in ck:
                for transaction in partition:
                    if freq_item.issubset(set(transaction)):
                        measure_support[str(freq_item)] += 1

    return measure_support.items()

# define time calculation
import time 
start_time = time.time()

# run data set
with open('/Users/wayoo/All My files/【Data Science】/CSC 440 Data Mining/Project/adult.csv') as file:
    adult = file.readlines()
transactions = []
for i in range(len(adult)):
    transactions.append(adult[i].strip().split(','))
total_transaction = len(transactions)
min_sup = len(transactions) * 0.5
len_of_partition = 1000  # define per partition length 
N = int(len(transactions)/len_of_partition)+1
P = partition_dataset(transactions, N, len_of_partition)
phase1 = phase1(P, min_sup)
all_candidate = combine(phase1, P)
phase2 = phase2(P, all_candidate)
Frequent_itemsets = []
for itemset, support in phase2:
    if support >= min_sup and itemset not in Frequent_itemsets:
        Frequent_itemsets.append(itemset)
print("The partition Apriori algorithm takes %s seconds." % (time.time() - start_time))
print(Frequent_itemsets)

############## Apriori##############################
import time 
from collections import defaultdict

# Define Apriori algorithm according to textbook method
def find_frequent_1_itemsets(D, min_sup):
    item_count = defaultdict(int)
    for transaction in D:
        for item in transaction:
            item_count[item] += 1
    L1 = []
    for item, count in item_count.items():
        if count >= min_sup:
            L1.append({item})
    return L1

def apriori_gen(Lk_1, k):
    Ck = []
    for l1 in Lk_1:
        for l2 in Lk_1:
            c = l1.union(l2)
            if len(c) == k:
                if c not in Ck and not has_infrequent_subset(c, Lk_1):
                    Ck.append(c)
    return Ck

def has_infrequent_subset(c, Lk_1):
    for x in c:
        s = c - set([x]) # s is subset s of c
        if s not in Lk_1:
            return True
        return False

def apriori(D, min_sup):
    k = 2
    L = []
    L0 = []
    L1 = find_frequent_1_itemsets(D, min_sup)
    L.append(L0)
    L.append(L1)
    while len(L[k-1]) != 0:
        Lk = []
        Ck = apriori_gen(L[k-1], k)
        c_count = defaultdict(int)
        for c in range(len(Ck)):
            for t in D:
                if Ck[c].issubset(t):
                    c_count[c] += 1
        for item, count in c_count.items():
            if count >= min_sup:
                Lk.append(Ck[item])
        L.append(Lk)
        k += 1
    L.remove(L0)
    return L
        

#####################Apriori###################################


The partition Apriori algorithm takes 2.172441244125366 seconds.
["{' White'}", "{' Male'}", "{' 0'}", "{' United-States'}", "{' <=50K'}", "{' Private'}", "{' White', ' Male'}", "{' White', ' 0'}", "{' United-States', ' White'}", "{' White', ' <=50K'}", "{' White', ' Private'}", "{' 0', ' Male'}", "{' United-States', ' Male'}", "{' United-States', ' 0'}", "{' 0', ' <=50K'}", "{' 0', ' Private'}", "{' United-States', ' <=50K'}", "{' United-States', ' Private'}", "{' Private', ' <=50K'}", "{' White', ' 0', ' Male'}", "{' United-States', ' White', ' Male'}", "{' United-States', ' White', ' 0'}", "{' White', ' 0', ' <=50K'}", "{' White', ' 0', ' Private'}", "{' United-States', ' White', ' <=50K'}", "{' United-States', ' White', ' Private'}", "{' United-States', ' 0', ' Male'}", "{' United-States', ' 0', ' <=50K'}", "{' United-States', ' 0', ' Private'}", "{' 0', ' Private', ' <=50K'}", "{' White', ' United-States', ' 0', ' Male'}", "{' White', ' <=50K', ' United-States', ' 0'}", "{' White'